# 4. Gt, Lt, Between, Within, Mean, Dedup, Where, Neq, Limit, Range, Skip, Tail, TimeLimit
#### By Kwan Yin Andrew Chau

In [1]:
import nbfinder
from loader import g
from loader import P
from loader import __
from loader import hkVertexId
from loader import sydneyVertexId

## gt(), lt()
__Get destinations from HK that have more than 5 runways__

In [2]:
g.V(hkVertexId).out('route').has('runways', P.gt(5)).valueMap('desc', 'runways').fold().next()

[{'runways': [7], 'desc': ['Dallas/Fort Worth International Airport']},
 {'runways': [6], 'desc': ['Boston Logan']},
 {'runways': [8], 'desc': ["Chicago O'Hare International Airport"]},
 {'runways': [6], 'desc': ['Amsterdam Airport Schiphol']}]

## between()
__Get destinations from HK that have 6 to 8 runways__

In [3]:
g.V(hkVertexId).out('route').has('runways', P.between(6, 8)).valueMap('desc', 'runways').fold().next()

[{'runways': [7], 'desc': ['Dallas/Fort Worth International Airport']},
 {'runways': [6], 'desc': ['Boston Logan']},
 {'runways': [6], 'desc': ['Amsterdam Airport Schiphol']}]

## within()
__Count of vertices that are within region "AU-NSW" and "AU-VIC"__

In [4]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).values('desc').fold().next()

['Sydney Kingsford Smith',
 'Albury Airport',
 'Moruya Airport',
 'Parkes Airport',
 'Orange Airport',
 'Narrandera Airport',
 'Warrnambool Airport',
 'Griffith Airport',
 'Melbourne Essendon Airport',
 'Newcastle Airport',
 'Armidale Airport',
 'Wagga Wagga City Airport',
 'Moree Airport',
 'Portland Airport',
 'Broken Hill Airport',
 'Lismore Airport',
 'Taree Airport',
 'Bathurst Airport',
 'Narrabri Airport',
 'Lord Howe Island Airport',
 'Coffs Harbour Airport',
 'Ballina Byron Gateway Airport',
 'Grafton Airport',
 'Dubbo City Regional Airport',
 'Gold Coast Airport',
 'Port Macquarie Airport',
 'Merimbula Airport',
 'Avalon Airport',
 'Tamworth Airport',
 'Mildura Airport',
 'Melbourne International Airport',
 'Sydney Bankstown Airport']

## mean()

__Get the mean between the number of runways of airports within regions "AU-NSW" and "AU-VIC"__

In [5]:
g.V().has('region', P.within('AU-NSW', 'AU-VIC')).values('runways').mean().next()

1.875

## dedup()

__Get airports that have destinations the HK airport does, and remove duplicates__

In [6]:
a = g.V(hkVertexId).out('route').in_('route').dedup().count().next()
g.V(hkVertexId).out('route').in_('route').values('desc').dedup().fold().next()

['Orlando International Airport',
 'Fort Lauderdale/Hollywood International Airport',
 'Baltimore/Washington International Airport',
 'Anchorage Ted Stevens',
 'Hartsfield - Jackson Atlanta International Airport',
 'Austin Bergstrom International Airport',
 'Washington Dulles International Airport',
 'Raleigh-Durham',
 'George Bush Intercontinental',
 'Miami International Airport',
 'New York John F. Kennedy International Airport',
 'Dallas/Fort Worth International Airport',
 'Nashville International Airport',
 'Minneapolis-St.Paul International Airport',
 'Seattle-Tacoma',
 'San Diego Lindbergh',
 'Tokyo Narita',
 'Palm Beach International Airport',
 'Boston Logan',
 'Denver International Airport',
 'El Paso International Airport',
 'Las Vegas Mc Carran',
 'Brisbane International Airport',
 'San Francisco International Airport',
 "Chicago O'Hare International Airport",
 'Ronald Reagan Washington National Airport',
 'Melbourne International Airport',
 'Toronto Pearson International Air

__Remove duplicates by number of runways__

In [7]:
g.V().has('region','AU-NSW').dedup().by('runways').values('code','runways').fold().next()

['SYD', 3, 'ABX', 1, 'MYA', 2, 'TMW', 4]

__Remove duplicates where there is the same A and C even if B is different__

In [8]:
g.V(hkVertexId).as_('a').V(sydneyVertexId).as_('c').both().as_('b').limit(10).dedup('a','c').select('a','b','c').fold().next()

[{'a': v[32776], 'b': v[4312], 'c': v[28712]}]

## where(), neq()

Exclude the HK airport itself

In [9]:
b = g.V(hkVertexId).as_('hk').out('route').in_('route').where(P.neq('hk')).dedup().count().next()
assert a == b + 1
b

1879

## limit()
Limit the amount of results to "n"

In [10]:
n = 5
count = g.V(hkVertexId).out('route').limit(n).count().next()
assert n == count
a = g.V(hkVertexId).out('route').limit(n).fold().next()
a

[v[4312], v[8312], v[8344], v[12328], v[12352]]

## range()
Equivalent to using a range starting from 0

In [11]:
b = g.V(hkVertexId).out('route').range(0, n).fold().next()
assert a == b

__Get the results from "n" to the end__

In [16]:
# This is not supported in python gremlin
# a = g.V(hkVertexId).out('route').range(n, -1).fold().next()

## skip()
Equivalent to skipping by "n"

In [13]:
# This is not supported in python gremlin
# a = g.V(hkVertexId).out('route').skip(n).fold().next()
# assert a == b

## tail()
__Get the last "n" results__

In [14]:
g.V(hkVertexId).out('route').tail(n).fold().next()

[v[1294456], v[1306744], v[1323128], v[1331320], v[2392184]]

There is no guarantee to the order of ranges, it is often based on order put into graph

## timeLimit()
Limit the query to however much can be processed within 10 ms

In [15]:
g.V(hkVertexId).repeat(__.timeLimit(10).out()).until(__.has('code','SYD')).path().by('code').next()

['HKG', 'SYD']